In [1]:
source('utils.r')
source('selinf_functions.r')
source('metrics.r')
source('data_generator.r')
source('cov_matrix.r')
library('lmmlasso')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack


Loading required package: lme4

Loading required package: mgcv

Loading required package: nlme


Attaching package: 'nlme'


The following object is masked from 'package:lme4':

    lmList


The following object is masked from 'package:dplyr':

    collapse


This is mgcv 1.8-42. For overview type 'help("mgcv-package")'.

This

----------------------------------------------------------------------
This is a test release of the package 'lmmlasso'. If you have any questions or problems, do not hesitate to contact the author.
----------------------------------------------------------------------


## Simulating random time
In this example we simulate data from a certain number of subjects and assume that time (introduced as a dummy variable) has a random effect. We assume that the fixed effect of timem is an important variable, and we don't include it in the penalization. We also try and estimate the variance-covariance matrix by using only the random components, or also including the fixed effect of time, and see if there are differences in the power results (TPR and average length of CIs).

In [2]:
set.seed(1)

n_subjects= 25
n_observations = 4
n = n_subjects * n_observations
p = 100
q <- n_observations-1
SNR = 4
prop_relevant = 0.1

data <- data_generator_random_time(n_subjects, n_observations, p, SNR, prop_relevant, rho=0.5)
X <- data$X
Z <- data$Z
subjects <- data$subjects
y <- data$y
beta <- data$beta
sd <- data$sd

## Old variance estimate

In [3]:
## Fixed lambda

lambda = 13

selFun <- function(y) selFun_fixed_lambda_randtime(X, Z, subjects, y, lambda)

sel <- selFun(y)
sel_vec <- sel$vec
sel_names <- sel$names
print(metrics(sel_vec,c(TRUE,beta!=0)))

$tpr
[1] 1

$fdr
[1] 0.5333333



In [4]:
## Adding Selective Inference
# Now we can define the function checking the congruency
# with the original selection
checkFun <- function(yb){

  all(selFun(yb)$vec == sel_vec)

}

sel_form = as.formula(
  paste("y ~ ",paste(sel_names[2:length(sel_names)], collapse='+'), "+ (t1 + t2 +t3|subjects)")
)

control <- lmerControl(
    check.nobs.vs.rankZ = "ignore",
    check.nobs.vs.nlev = "ignore",
    check.nlev.gtreq.5 = "ignore",
    check.nlev.gtr.1 = "ignore",
    check.nobs.vs.nRE= "ignore",
)


final_model = lmer(formula = sel_form, control= control ,data=data.frame(X, subjects, y))

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"Model failed to converge with 1 negative eigenvalue: -6.5e-04"


In [5]:
final_model

Linear mixed model fit by REML ['lmerModLmerTest']
Formula: sel_form
   Data: data.frame(X, subjects, y)
REML criterion at convergence: 322.8462
Random effects:
 Groups   Name        Std.Dev. Corr             
 subjects (Intercept) 0.9939                    
          t1          1.1804   -0.24            
          t2          0.7723    0.01 -0.34      
          t3          0.4639   -0.04 -0.64 -0.45
 Residual             0.3639                    
Number of obs: 100, groups:  subjects, 25
Fixed Effects:
(Intercept)           t1           t2           t3           X1           X2  
    0.30987     -0.65235     -1.13769     -0.26843     -0.91508     -0.74776  
         X3           X4           X5           X6           X7           X8  
    0.76937     -1.11749     -0.65624     -0.74253     -0.95389     -0.74552  
         X9          X10          X15          X18          X22          X28  
   -0.79461     -0.88041     -0.09179      0.12788     -0.21561      0.05211  
        X40   

In [10]:
vec2mlist(Cv_to_Vv(getME(final_model,'theta')))

7.46152259,-2.126359,0.07071013,-0.1393707
-2.12635904,10.524733,-2.35437938,-2.6366907
0.07071013,-2.354379,4.50526983,-1.2095839
-0.13937067,-2.636691,-1.20958394,1.6258551


In [6]:
res <- mocasin(final_model, this_y = y, conditional = FALSE, varForSampling = 'minMod',
               checkFun = checkFun, nrSamples = 50)
# create a boolean vector for the ones selected controlling fdr level with BH procedure

sel_with_selinf <- selection_with_selinf(res, sel_vec, fdr_level = 0.1)
metrics(sel_with_selinf,c(1,beta!=0))

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?"


Computing inference for variable (location)  1 

  |======================================================================| 100%


Computing inference for variable (location)  2 

  |======================================================================| 100%


Computing inference for variable (location)  3 

  |======================================================================| 100%


Computing inference for variable (location)  4 

  |======================================================================| 100%


Computing inference for variable (location)  5 

  |======================================================================| 100%


Computing inference for variable (location)  6 

  |======================================================================| 100%


Computing inference for variable (location)  7 

  |======================================================================| 100%


Computing inference for variable (location)  8 

  |===============================

Warning message in selinf(survr = survr, tstat = tstat, w = w, var_est = var_est, :
"Lower interval limit is not the 2.5%-quantile, but the (0.00833)%-quantile."




Computing inference for variable (location)  9 

  |======================================================================| 100%


Computing inference for variable (location)  10 

  |======================================================================| 100%


Computing inference for variable (location)  11 

  |======================================================================| 100%


Computing inference for variable (location)  12 

  |======================================================================| 100%


Computing inference for variable (location)  13 

  |======================================================================| 100%


Computing inference for variable (location)  14 

  |======================================================================| 100%


Computing inference for variable (location)  15 

  |======================================================================| 100%


Computing inference for variable (location)  16 

  |======================

$tpr
[1] 0.5714286

$fdr
[1] 0.4285714

In [7]:
ci_length(res)

[1] 2.91259

## New variance estimate

In [15]:
source('minMod_modif.r')

In [11]:
## Fixed lambda

lambda = 13

selFun <- function(y) selFun_fixed_lambda_randtime(X, Z, subjects, y, lambda)

sel <- selFun(y)
sel_vec <- sel$vec
sel_names <- sel$names
print(metrics(sel_vec,c(TRUE,beta!=0)))

$tpr
[1] 1

$fdr
[1] 0.5333333



In [12]:
## Adding Selective Inference
# Now we can define the function checking the congruency
# with the original selection
checkFun <- function(yb){

  all(selFun(yb)$vec == sel_vec)

}

sel_form = as.formula(
  paste("y ~ ",paste(sel_names[2:length(sel_names)], collapse='+'), "+ (t1 + t2 +t3|subjects)")
)

control <- lmerControl(
    check.nobs.vs.rankZ = "ignore",
    check.nobs.vs.nlev = "ignore",
    check.nlev.gtreq.5 = "ignore",
    check.nlev.gtr.1 = "ignore",
    check.nobs.vs.nRE= "ignore",
)


final_model = lmer(formula = sel_form, control= control ,data=data.frame(X, subjects, y))

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"Model failed to converge with 1 negative eigenvalue: -6.5e-04"


In [16]:
res <- mocasin(final_model, this_y = y, conditional = FALSE,
               checkFun = checkFun, nrSamples = 50)
# create a boolean vector for the ones selected controlling fdr level with BH procedure

sel_with_selinf <- selection_with_selinf(res, sel_vec, fdr_level = 0.1)
metrics(sel_with_selinf,c(1,beta!=0))

Computing inference for variable (location)  1 

  |                                                                      |   0%

  |========================                                              |  34%

: 

In [14]:
ci_length(res)

[1] 2.91259